In [26]:
from collections import defaultdict
from nltk import word_tokenize
from math import log, sqrt

In [ ]:

class kNN:
    def __init__(tokenizer=None):
        self.params = {}
        self.tokenizer = tokenizer if not tokenizer == None else word_tokenizer
        
    def fit(self, X, y):
        # check valid dataset
        assert len(X) == len(y)
        
        # save originals
        self.X = X
        self.y = y
        self.doc2idx = lambda d: X.index(d)
        self.idx2doc = lambda i: X[i]
        
        # TDM
        self.vocabulary = list()
        self.posting = list()
        self.lexicon = defaultdict(lambda :-1)
        self.docInfo = defaultdict(lambda: {"maxTf":0, "vecLen":0.0})
        for doc in X:
            localPosting = defaultdict(int)
            for token in self.tokenizer(doc):
                if token not in self.vocabulary:
                    self.vocabulary.append(token)
                localPosting[token] += 1
                
            maxTf = max(localPosting.values())
            docID = self.doc2idx(doc)
            self.docInfo[docID]["maxTf"] = maxTf
            
            for token, freq in localPosting.items():
                ptr = self.lexicon[token]
                nexPtr = len(self.posting)
                self.posting.append((docID, freq, ptr))
                self.lexicon[token] = nextptr
        
        # weight
        self.weight = 
                
        # distance
        self 
        
    def predict_prop(self, test, k=5, method="euclide"):
        # test indexing
        
    
    
    def predict(self, test, k=5, method="euclide"):
        # predict
        
        
        
        
        # sorting
        
    def predict_many(self, testset, k=5, method="euclide"):
        